In [ ]:
#Read data from files and prepare for analysis
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#Read first 8 columns of the flight data file
df = pd.read_csv('329994478_72017_3649_airline_delay_causes.csv').iloc[:,:8]

#Function to analyse delays
def ontime(df) :
    x=df[['carrier_name','arr_flights','arr_del15']].groupby('carrier_name').sum()
    x['on_time_%'] = np.round(100*(1 - (x.arr_del15/x.arr_flights)),2)
    return x.sort_values(by='on_time_%',ascending=False)

In [ ]:
%%javascript
class XTable extends HTMLElement {
    constructor() {
        super() ;
        if ( this.sroot == undefined )
            this.sroot = this.attachShadow({mode:'open'}) ;
    }
    connectedCallback() {
        this.sroot.innerHTML = ''
    }
    static get observedAttributes() { return ['data-command','data-style'] ; }
    attributeChangedCallback(name,oldValue,newValue) {
        if ( name == 'data-style' ) {
            if ( this._style ) {
                alert('Style can be set only once') ;
                return
            }
            this._style = '<style>' + newValue + '</style>';
            return ;
        }
        if ( newValue == undefined || newValue.length == 0 )
            return ;
        this._command = newValue ;
        this._commandChanged() ;
    }
    _commandChanged() {
        if ( this._command == undefined || this._command.length == 0 )
            return ;
        var container = this ;
        IPython.notebook.kernel.execute( this._command, {
            iopub : {
                output : function(res) {
                    container.sroot.innerHTML = ''
                    if ( container.style )
                        container.sroot.innerHTML += container._style ;
                    if (res.content.evalue) {
                        container.sroot.innerHTML += res.content.traceback;
                        return ;
                    }
                    if ( res.content.data['image/png'] ) {
                        container.sroot.innerHTML += '<img src="data:image/png;base64,' + res.content.data['image/png'] +'">' ;
                        return ;
                    }
                    if ( res.msg_type == 'execute_result' )
                        if ( res.content.data['text/html'] ) {
                            container.sroot.innerHTML += res.content.data['text/html'];
                        }
                    var table = container.sroot.querySelector('table') ; //Assume only one table for now
                    if ( table )
                        table.setAttribute('border',0) ;
                }
            }
        }, {
            silent : false
        })
    }

}
(function() {
    if ( window.customElements == undefined ) {
        alert('Unsupported browser') ;
        return ;
    }
    if ( customElements.get('x-table') == undefined )
        customElements.define('x-table',XTable) ;
})() ;

In [ ]:
%%html
Select an option <select onchange='xtable_01.setAttribute("data-command",this.value);'>
    <option value=''></option>
    <option value='ontime(df)'>On time comparison</option>
    <option value='p=ontime(df)["on_time_%"].plot(kind="bar",figsize=(14,5));plt.plot();'>On time graph</option>
</select>
Or search for :
Airline Code <input type='text' size='3' 
    onchange='xtable_01.setAttribute("data-command", 
        "df[df.carrier.str.contains(\""+this.value+"\",case=False)]")'> 
Airport Name <input type='text' size='30' 
    onchange='xtable_01.setAttribute("data-command", 
    "df[df.airport_name.str.contains(\""+this.value+"\",case=False)].sort_values(by=\"arr_flights\",ascending=False)")'> 
<hr>
<div style='border: solid 1px lightgray;min-height:300px;'>
    <x-table id='xtable_01' data-command='' data-style='td {font-size:9pt}'></x-table>
</div>